In [1]:
"""
chatbot_nlp.py

A simple NLP-based chatbot using NLTK for preprocessing and scikit-learn
for intent classification.

Features:
- Tokenization, stopword removal, lemmatization (NLTK)
- TF-IDF vectorizer + LogisticRegression classifier
- Simple rule-based fallback and response selection
- Save / load trained model + vectorizer
- Interactive CLI chat loop

How to run:
1. pip install nltk scikit-learn joblib
2. python chatbot_nlp.py
3. (First run will download small NLTK datasets)

Extend:
- Add more intents/responses in the INTENTS list
- Replace classifier with neural models for more complex behavior
"""

import json
import random
import re
import os
from typing import List, Tuple, Dict

# NLP & ML libraries
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import joblib

In [2]:
# -- Ensure necessary NLTK data is downloaded (only first run) --
nltk_packages = ["punkt", "wordnet", "omw-1.4", "stopwords", "punkt_tab"]
for pkg in nltk_packages:
    try:
        nltk.data.find(pkg)
    except LookupError:
        nltk.download(pkg)

# ------------------------------
# 1) Example intents dataset
# ------------------------------
# This is a small sample. Add more examples to improve accuracy.
INTENTS = [
    {
        "tag": "greeting",
        "patterns": [
            "hello", "hi", "hey", "good morning", "good evening",
            "what's up", "yo", "howdy", "hiya"
        ],
        "responses": [
            "Hello! How can I help you today?",
            "Hi there — what can I do for you?",
            "Hey! Need any help?"
        ]
    },
    {
        "tag": "goodbye",
        "patterns": ["bye", "goodbye", "see you", "see ya", "talk later"],
        "responses": ["Goodbye! Have a great day.", "See you later!", "Bye — come back anytime!"]
    },
    {
        "tag": "thanks",
        "patterns": ["thanks", "thank you", "much appreciated", "thx"],
        "responses": ["You're welcome!", "No problem!", "Anytime 😊"]
    },
    {
        "tag": "about_bot",
        "patterns": ["who are you", "what are you", "your name", "about you"],
        "responses": [
            "I'm a simple chatbot built with NLTK and scikit-learn.",
            "I am an NLP chatbot created to demonstrate intent classification."
        ]
    },
    {
        "tag": "help",
        "patterns": ["I need help", "can you help me", "assist me", "i need assistance"],
        "responses": [
            "Sure — tell me what you need help with.",
            "I'm here to help. What would you like to do?"
        ]
    },
    {
        "tag": "weather_query",
        "patterns": [
            "what's the weather", "tell me the weather", "is it raining",
            "how's the weather", "weather today"
        ],
        "responses": [
            "I can't fetch live weather yet, but you can integrate a weather API such as OpenWeatherMap.",
            "I don't have live weather access in this demo — add an API to enable that."
        ]
    },
    {
        "tag": "time_query",
        "patterns": ["what time is it", "tell me the time", "current time", "time now"],
        "responses": [
            "I can tell the local time if you add a timezone utility.",
            "Time-check feature isn't enabled in this demo. Add Python's datetime to respond."
        ]
    },
    {
        "tag": "smalltalk_name",
        "patterns": ["what's your favorite color", "do you like music", "are you a human"],
        "responses": [
            "I don't have personal preferences, but I like helping you!",
            "I'm not human — just code and models."
        ]
    }
]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
# ------------------------------
# 2) NLP Preprocessing helpers
# ------------------------------
lemmatizer = WordNetLemmatizer()
STOPWORDS = set(stopwords.words("english"))


def normalize_text(text: str) -> str:
    """
    Clean + normalize text:
    - lowercase
    - remove non-alphanumeric characters except spaces
    - tokenize, remove stopwords, lemmatize
    - join back into a string
    This function will be used inside the TfidfVectorizer via a tokenizer wrapper.
    """
    # lowercase
    text = text.lower()

    # remove URLs and emails (simple)
    text = re.sub(r"http\S+|www\S+|@\S+", " ", text)

    # remove punctuation (keep spaces & alphanumerics)
    text = re.sub(r"[^a-z0-9\s]", " ", text)

    # tokenize using nltk
    tokens = nltk.word_tokenize(text)

    # remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in STOPWORDS and tok.isalpha()]

    return " ".join(tokens)


def simple_tokenizer(text: str) -> List[str]:
    """
    Tokenizer wrapper expected by TfidfVectorizer: returns list of tokens
    after normalization. scikit-learn expects a callable tokenizer that returns tokens.
    """
    normalized = normalize_text(text)
    return normalized.split()

In [4]:
# ------------------------------
# 3) Prepare dataset for training
# ------------------------------
def build_dataset(intents: List[Dict]) -> Tuple[List[str], List[str]]:
    """
    Convert the INTENTS structure into X (text samples) and y (tags).
    Each pattern becomes a training sample.
    """
    X = []
    y = []
    for intent in intents:
        tag = intent["tag"]
        for pattern in intent["patterns"]:
            X.append(pattern)
            y.append(tag)
    return X, y

In [5]:
# ------------------------------
# 4) Build & train a classifier pipeline
# ------------------------------
def build_pipeline() -> Pipeline:
    """
    Build a scikit-learn Pipeline:
    - TfidfVectorizer with the simple_tokenizer
    - LogisticRegression classifier for multiclass intent classification
    """
    tfidf = TfidfVectorizer(tokenizer=simple_tokenizer, ngram_range=(1, 2))
    clf = LogisticRegression(max_iter=500)
    pipeline = Pipeline([("tfidf", tfidf), ("clf", clf)])
    return pipeline


def train_pipeline(pipeline: Pipeline, X: List[str], y: List[str]) -> Pipeline:
    """
    Train the pipeline. Splits for a quick validation report.
    Returns trained pipeline.
    """
    if len(X) < 2:
        raise ValueError("Need at least 2 training samples")

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    pipeline.fit(X_train, y_train)

    # Quick evaluation
    y_pred = pipeline.predict(X_test)
    print("\n--- Training evaluation (quick) ---")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

    return pipeline

In [6]:
# ------------------------------
# 5) Response selection & chat logic
# ------------------------------
# Load a mapping of tag -> responses for quick lookup
TAG_TO_RESPONSES = {intent["tag"]: intent["responses"] for intent in INTENTS}


def get_response_for_tag(tag: str) -> str:
    """
    Choose a random response from the list for the predicted tag.
    """
    responses = TAG_TO_RESPONSES.get(tag, [])
    if responses:
        return random.choice(responses)
    return "Sorry, I don't have a response for that."


def predict_intent(pipeline: Pipeline, message: str, threshold: float = 0.5) -> Tuple[str, float]:
    """
    Predict intent tag for a given message.
    - Returns (tag, probability)
    - If highest probability < threshold, returns ('fallback', probability)
    """
    # pipeline.predict_proba returns probabilities for classes
    proba = pipeline.predict_proba([message])[0]  # shape = (n_classes, )
    classes = pipeline.classes_
    best_idx = proba.argmax()
    best_tag = classes[best_idx]
    best_prob = proba[best_idx]
    if best_prob < threshold:
        return "fallback", best_prob
    return best_tag, best_prob


def fallback_response() -> str:
    """
    Generic fallback when model is not confident.
    """
    return "Sorry, I didn't understand that. Can you rephrase or ask something else?"

In [7]:
# ------------------------------
# 6) Save / Load helpers
# ------------------------------
def save_model(pipeline: Pipeline, filepath: str):
    joblib.dump(pipeline, filepath)
    print(f"Model saved to {filepath}")


def load_model(filepath: str) -> Pipeline:
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"No model found at {filepath}")
    print(f"Loading model from {filepath}")
    return joblib.load(filepath)

In [8]:
# ------------------------------
# 7) Interactive chat loop
# ------------------------------
def chat_loop(pipeline: Pipeline):
    """
    Simple CLI chat loop. Type 'quit' or 'exit' to end.
    """
    print("Chatbot is ready! (type 'quit' or 'exit' to stop)\n")
    while True:
        message = input("You: ").strip()
        if not message:
            continue
        if message.lower() in ("quit", "exit"):
            print("Bot: Goodbye! 👋")
            break

        # optional: handle simple rule-based patterns before ML (numbers, greetings, etc)
        # but our classifier already covers greetings

        tag, prob = predict_intent(pipeline, message, threshold=0.3)
        if tag == "fallback":
            print(f"Bot: {fallback_response()} (confidence={prob:.2f})")
        else:
            response = get_response_for_tag(tag)
            print(f"Bot: {response} (intent='{tag}', confidence={prob:.2f})")

In [ ]:
# ------------------------------
# 8) Main: train (or load) and run
# ------------------------------
MODEL_FILE = "chatbot_pipeline.joblib"


def main():
    # Build dataset
    X, y = build_dataset(INTENTS)

    # If a saved model exists, load it; otherwise train a new one.
    if os.path.exists(MODEL_FILE):
        pipeline = load_model(MODEL_FILE)
    else:
        pipeline = build_pipeline()
        pipeline = train_pipeline(pipeline, X, y)
        save_model(pipeline, MODEL_FILE)

    # Start interactive chat
    chat_loop(pipeline)


if __name__ == "__main__":
    main()


Loading model from chatbot_pipeline.joblib
Chatbot is ready! (type 'quit' or 'exit' to stop)

You: Hi
Bot: Sorry, I didn't understand that. Can you rephrase or ask something else? (confidence=0.24)
You: hi
Bot: Sorry, I didn't understand that. Can you rephrase or ask something else? (confidence=0.24)
You: hello
Bot: Hey! Need any help? (intent='greeting', confidence=0.39)
You: who are you
Bot: Sorry, I didn't understand that. Can you rephrase or ask something else? (confidence=0.24)
You: who are you?
Bot: Sorry, I didn't understand that. Can you rephrase or ask something else? (confidence=0.24)
